In [4]:
import pandas as pd
import re
import json

In [5]:
# IBRA
ibra = pd.read_csv('data/ibra.csv')
ibra_bioregions = pd.read_csv('data/ibra_bioregions.csv')

# IUCN
bioregion_by_IUCN = pd.read_csv('data/bioregion_by_IUCN.csv')
IUCN_category = pd.read_csv('data/IUCN_category.csv')

### IBRA

In [6]:
ibra.head()

,IBRA Region Code,total_ce
0,ARC,1
1,ARP,2
2,AUA,11
3,AVW,16
4,BBN,5


In [7]:
ibra_bioregions.head()

,IBRA Region Name,IBRA Region Code,Jurisdiction(s),IBRA Area (ha),1 Area Protected (ha),% IBRA Region Protected,2 Contribution to NRS (%)
0,Arnhem Coast,ARC,NT,"3,335,669","1,779,616",53.35,1.17
1,Arnhem Plateau,ARP,NT,"2,306,023","1,728,677",74.96,1.14
2,Australian Alps,AUA,"ACT, NSW, VIC","1,232,981","788,727",63.97,0.52
3,Avon Wheatbelt,AVW,WA,"9,517,104","375,948",3.95,0.25
4,Ben Lomond,BEL,TAS,"657,500","106,341",16.17,0.07


In [8]:
ibra_bioregions_w_total_ce = pd.merge(ibra_bioregions, ibra, on="IBRA Region Code")
ibra_bioregions_w_total_ce.head()

,IBRA Region Name,IBRA Region Code,Jurisdiction(s),IBRA Area (ha),1 Area Protected (ha),% IBRA Region Protected,2 Contribution to NRS (%),total_ce
0,Arnhem Coast,ARC,NT,"3,335,669","1,779,616",53.35,1.17,1
1,Arnhem Plateau,ARP,NT,"2,306,023","1,728,677",74.96,1.14,2
2,Australian Alps,AUA,"ACT, NSW, VIC","1,232,981","788,727",63.97,0.52,11
3,Avon Wheatbelt,AVW,WA,"9,517,104","375,948",3.95,0.25,16
4,Ben Lomond,BEL,TAS,"657,500","106,341",16.17,0.07,7


### IUCN

In [9]:
bioregion_by_IUCN.head()

,IBRA Region Name,IBRA Region Code,IA,IB,II,III,IV,I-IV Total,% IBRA Region,V,VI,V-VI Total,% IBRA Region.1,Not Applicable,Not Assigned,Total Area Protected (ha),IBRA Region Area (ha),% IBRA Region protected
0,Arnhem Coast,ARC,NaN,NaN,NaN,NaN,NaN,-,0.00,"396,725","1,382,891","1,779,616",53.35,NaN,NaN,"1,779,616","3,335,669",53.35
1,Arnhem Plateau,ARP,NaN,NaN,"486,415",NaN,NaN,"486,415",21.09,NaN,"1,242,262","1,242,262",53.87,NaN,NaN,"1,728,677","2,306,023",74.96
2,Australian Alps,AUA,"11,079","27,674","738,549","10,296",244,"787,842",63.90,0,886,886,0.07,NaN,NaN,"788,727","1,232,981",63.97
3,Avon Wheatbelt,AVW,"163,602",NaN,"200,779",NaN,165,"364,547",3.83,NaN,"4,850","4,850",0.05,NaN,"6,552","375,948","9,517,104",3.95
4,Ben Lomond,BEL,28,NaN,"19,950","5,820","60,610","86,408",13.14,601,"19,333","19,933",3.03,NaN,NaN,"106,341","657,500",16.17


In [10]:
IUCN_category.head()

,IUCN Category,1 Number,2 Area (ha),3 Average size (ha),4 % of Australia,Contribution to NRS (%)
0,IA,"2,541","15,966,348","6,283",2.07,10.52
1,IB,65,"3,846,201","59,172",0.50,2.53
2,II,"1,084","38,096,535","35,144",4.96,25.10
3,III,"2,375","1,865,843",786,0.24,1.23
4,IV,"4,195","2,274,848",542,0.30,1.50


In [11]:
ibra_bioregions_w_total_ce.to_csv('data/ibra_bioregions_w_total_ce.csv')

### Scripting that reformats data for sunburst graph

In [12]:
with open('data/ibra_bioregions.csv') as opener:
    data = opener.readlines()
    header = data[0].strip().split(',')
    data = data[1:]
    # too ugly
    data = [re.sub(r'(" \d+)\,', r'\1', i).strip() for i in data]
    data = [re.sub(r'(" \d+)\,', r'\1', i) for i in data]
    data = [re.sub(r'(" \d+)\,', r'\1', i) for i in data]
    data = [re.sub(r'"', r'', i) for i in data]
    data_ungroup = []
    for d in data:
        values = d.split(',')
        if values[2] != '-':
            first = values[:2]
            last = [float(j) for j in values[-4:]]
            states = [s.strip() for s in values[2: -4]] 
            for state in states:
                data_ungroup.append(first + [state] + last)

In [13]:
data_groupby_state = dict()
for d in data_ungroup:
    if d[2] not in data_groupby_state:
        data_groupby_state[d[2]] = [d[: 2] + d[3:]]
    else:
        data_groupby_state[d[2]].append(d[: 2] + d[3:])

In [14]:
data_json = {'name': 'flare', 'children': []}

for state, values in data_groupby_state.items():
    data_json['children'].append({
        'name': state,
        'children': [
            {
                'name': v[0], 
                'children': [
                    {
                        'name': f'Contribution to NRS(%): {v[-1]}',
                        'value': v[-1]//2
                    },
                    {
                        'name': f'IBRA Region Protected(%): {v[-2]}',
                        'value': v[-1]//2
                    }
                ]
            } for v in values]
    })

with open('data/sunburst_graph_data.json', 'w') as outfile:
    json.dump(data_json, outfile, indent=2)